## Parsing Tweet Data into One Line Table

analyzing tweet connection from tweet, retweet, and replies
this notebook is intended to create social network analysis and how many users are interaction to each other

In [1]:
import os
import re
import glob
import json
import pickle
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.options.display.max_rows = 10

In [3]:
# tweet_file_paths = glob.glob("../data/7K_INDO/*.json")
tweet_file_paths = glob.glob("../data/user_timeline_46K/*.json")

In [4]:
len(tweet_file_paths)

46099

In [5]:
def read_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        try:
            return json.load(f)
        except:
            return []

In [6]:
def parsing_tweet(tweet):
    """Parsing tweet
    """
    screen_name = tweet["user"]["screen_name"]
    id_tweet = tweet["id_str"]
#     full_text = tweet["full_text"]
    full_text = tweet["text"]
    hashtags = tweet["entities"]["hashtags"]
    user_mentions = tweet["entities"]["user_mentions"]
    created_at = tweet["created_at"]
    
    parsed = {"screen_name": screen_name, "id_tweet": id_tweet, "full_text": full_text, "hashtags": hashtags,
           "user_mentions": user_mentions, "created_at": created_at}
    
    if tweet["in_reply_to_status_id_str"]:
        parsed["in_reply_to_status_id_str"] = tweet["in_reply_to_status_id_str"]
        parsed["in_reply_to_user_id_str"] = tweet["in_reply_to_user_id_str"]
        parsed["in_reply_to_screen_name"] = tweet["in_reply_to_screen_name"]
        
    
    if tweet["is_quote_status"]:
        parsed["quote_is_quote_status"] = tweet["is_quote_status"]
        try:
            parsed["quote_screen_name"] = tweet["quoted_status"]["user"]["screen_name"]
        except:
            pass
        try:
            parsed["quote_id_tweet"] = tweet["quoted_status"]["id_str"]
        except:
            pass
        try:
            parsed["quote_full_text"] = tweet["quoted_status"]["full_text"]
        except:
            pass
        try:
            parsed["quote_hashtags"] = tweet["quoted_status"]["entities"]["hashtags"]
        except:
            pass
        try:
            parsed["quote_user_mentions"] = tweet["quoted_status"]["entities"]["user_mentions"]
        except:
            pass
        try:
            parsed["quote_created_at"] = tweet["quoted_status"]["created_at"]
        except:
            pass
    return parsed 

In [ ]:
list_of_parsed_data = []
for path in tqdm(tweet_file_paths):
    data = read_json(path)
    break
    list_of_parsed_data.extend(list(map(parsing_tweet, data["tweets"])))
    list_of_parsed_data.extend(list(map(parsing_tweet, data["retweets"])))
    list_of_parsed_data.extend(list(map(parsing_tweet, data["replies"])))

In [10]:
list_of_parsed_data = []
for idx, path in tqdm(enumerate(tweet_file_paths[46001:], 46001)):
    data = read_json(path)
    list_of_parsed_data.extend(list(map(parsing_tweet, data)))
    
json.dump(list_of_parsed_data, open(f"../data/user_timeline_46K_parsed/{idx}.json", 'w'))

98it [00:03, 32.04it/s]


In [ ]:
pd.DataFrame(list_of_parsed_data)

## parsed tweet to dataframe

In [ ]:
column_ordered = ['screen_name', 'id_tweet', 'full_text', 'hashtags', 'user_mentions', 'created_at',
                 'quote_is_quote_status', 'quote_screen_name', 'quote_id_tweet', 'quote_full_text',
                  'quote_hashtags', 'quote_user_mentions', 'quote_created_at', 
                  "in_reply_to_status_id_str", "in_reply_to_user_id_str", "in_reply_to_screen_name"]

In [ ]:
d_tweets = pd.DataFrame(list_of_parsed_data)

In [ ]:
d_tweets = d_tweets[column_ordered]

In [ ]:
d_tweets.shape

In [ ]:
d_tweets.head()

Save file

In [ ]:
with open("../data/supports/parsed_7003.pkl", "wb") as file:
    pickle.dump(d_tweets, file)